In [2]:
import almasim.astro as uas
import numpy as np 
import pandas as pd 
import astropy.units as U
import almasim.alma as ual
import os
from astropy.constants import c
from astropy.cosmology import FlatLambdaCDM
import math
from math import pi, ceil


In [4]:
def remove_non_numeric(text):
        """Removes non-numeric characters from a string.
        Args:
            text: The string to process.

        Returns:
            A new string containing only numeric characters and the decimal point (.).
        """
        numbers = "0123456789."
        return "".join(char for char in text if char in numbers)

def freq_supp_extractor(freq_sup, obs_freq):
        freq_band, n_channels, freq_mins, freq_maxs, freq_ds = [], [], [], [], []
        freq_sup = freq_sup.split("U")
        for i in range(len(freq_sup)):
            sup = freq_sup[i][1:-1].split(",")
            sup = [su.split("..") for su in sup][:2]
            freq_min, freq_max = float(remove_non_numeric(sup[0][0])), float(
                remove_non_numeric(sup[0][1])
            )
            freq_d = float(remove_non_numeric(sup[1][0]))
            freq_min = freq_min * U.GHz
            freq_max = freq_max * U.GHz
            freq_d = freq_d * U.kHz
            freq_d = freq_d.to(U.GHz)
            freq_b = freq_max - freq_min
            n_chan = int(freq_b / freq_d)
            freq_band.append(freq_b)
            n_channels.append(n_chan)
            freq_mins.append(freq_min)
            freq_maxs.append(freq_max)
            freq_ds.append(freq_d)
        freq_ranges = np.array(
            [[freq_mins[i].value, freq_maxs[i].value] for i in range(len(freq_mins))]
        )
        idx_ = np.argwhere(
            (obs_freq.value >= freq_ranges[:, 0])
            & (obs_freq.value <= freq_ranges[:, 1])
        )[0][0]
        freq_range = freq_ranges[idx_]
        band_range = freq_range[1] - freq_range[0]
        n_channels = n_channels[idx_]
        central_freq = freq_range[0] + band_range / 2
        freq_d = freq_ds[idx_]
        return band_range * U.GHz, central_freq * U.GHz, n_channels, freq_d

def cont_finder(cont_frequencies, line_frequency):
        # cont_frequencies=sed['GHz'].values
        distances = np.abs(
            cont_frequencies - np.ones(len(cont_frequencies)) * line_frequency
        )
        return np.argmin(distances)

def normalize_sed(
    sed,
    lum_infrared,
    solid_angle,
    cont_sens,
    freq_min,
    freq_max,
    remote=False,
):
    so_to_erg_s = 3.846e33  # Solar luminosity to erg/s -XX
    lum_infrared_erg_s = lum_infrared * so_to_erg_s  # luminosity in erg/s -XX
    sed["Jy"] = lum_infrared_erg_s * sed["erg/s/Hz"] * 1e23 / solid_angle
    cont_mask = (sed["GHz"].values >= freq_min) & (sed["GHz"].values <= freq_max)
    if sum(cont_mask) > 0:
        cont_fluxes = sed["Jy"].values[cont_mask]
        min_ = np.min(cont_fluxes)
    else:
        freq_point = np.argmin(np.abs(sed["GHz"].values - freq_min))
        cont_fluxes = sed["Jy"].values[freq_point]
        min_ = cont_fluxes
    lum_save = lum_infrared
    while min_ > cont_sens:
        lum_infrared -= 0.1 * lum_infrared
        lum_infrared_erg_s = so_to_erg_s * lum_infrared
        sed["Jy"] = lum_infrared_erg_s * sed["erg/s/Hz"] * 1e23 / solid_angle
        cont_mask = (sed["GHz"] >= freq_min) & (sed["GHz"] <= freq_max)
        if sum(cont_mask) > 0:
            cont_fluxes = sed["Jy"].values[cont_mask]
            min_ = np.min(cont_fluxes)
        else:
            freq_point = np.argmin(np.abs(sed["GHz"].values - freq_min))
            cont_fluxes = sed["Jy"].values[freq_point]
            min_ = cont_fluxes

    return sed, lum_infrared_erg_s, lum_infrared
def sed_reading(
    type_,
    path,
    cont_sens,
    freq_min,
    freq_max,
    remote,
    lum_infrared=None,
    redshift=None,
):
    cosmo = FlatLambdaCDM(H0=70 * U.km / U.s / U.Mpc, Tcmb0=2.725 * U.K, Om0=0.3)
    if (
        type_ == "extended"
        or type_ == "diffuse"
        or type_ == "molecular"
        or type_ == "galaxy-zoo"
        or type_ == "hubble-100"
    ):
        file_path = os.path.join(path, "SED_low_z_warm_star_forming_galaxy.dat")
        if redshift is None:
            redshift = 10 ** (-4)
        if lum_infrared is None:
            lum_infrared = 1e12  # luminosity in solar luminosities
    elif type_ == "point" or type_ == "gaussian":
        file_path = os.path.join(path, "SED_low_z_type2_AGN.dat")
        if redshift is None:
            redshift = 0.05
        if lum_infrared is None:
            lum_infrared = 1e12  # luminosity in solar luminosities
    else:
        return "Not valid type"
    # L (erg/s/Hz) = 4 pi d^2(cm) * 10^-23 Flux (Jy)
    #  Flux (Jy) =L (erg/s/Hz) * 10^23 /  * 4 pi d^2(cm)
    # To normalize we multiply by lum_infrared_jy
    distance_Mpc = cosmo.luminosity_distance(redshift).value  # distance in Mpc
    Mpc_to_cm = 3.086e24  # Mpc to cm
    distance_cm = distance_Mpc * Mpc_to_cm  # distance in cm  -XX
    solid_angle = 4 * pi * distance_cm**2  # solid angle in cm^2 -XX
    # Load the SED
    sed = pd.read_csv(file_path, sep=r"\s+")
    # Convert to GHz
    sed["GHz"] = sed["um"].apply(
        lambda x: (x * U.um).to(U.GHz, equivalencies=U.spectral()).value
    )
    # Re normalize the SED and convert to Jy from erg/s/Hz
    sed, lum_infrared_erg_s, lum_infrared = normalize_sed(
        sed, lum_infrared, solid_angle, cont_sens, freq_min, freq_max, remote
    )
    #  Flux (Jy) =L (erg/s/Hz) * 10^23 /  * 4 pi d^2(cm)
    flux_infrared = lum_infrared_erg_s * 1e23 / solid_angle  # Jy * Hz
    # flux_infrared_jy = flux_infrared  / (sed['GHz'].values *
    # U.GHz).to(U.Hz).value  # Jy
    sed.drop(columns=["um", "erg/s/Hz"], inplace=True)
    sed = sed.sort_values(by="GHz", ascending=True)
    return sed, flux_infrared, lum_infrared

In [15]:
main_path = '/Users/michele/GitHub/ALMASim/almasim'
sim_output_dir = '/Users/michele/GitHub/ALMASim/almasim/experimental/'
rest_freq, line_names = uas.get_line_info(main_path)
metadata = pd.read_csv(main_path + '/metadata/qso_metadata.csv')
metadata = metadata.iloc[0]
freq_support = metadata['Freq.sup.']
source_freq = metadata['Freq'] * U.GHz
cont_sens = metadata['Cont_sens_mJybeam']
antenna_array = metadata['antenna_arrays']
band_range, central_freq, t_channels, delta_freq = freq_supp_extractor(
            freq_support, source_freq
        )

ual.generate_antenna_config_file_from_antenna_array(
            antenna_array, main_path, sim_output_dir
        )
antennalist = os.path.join(sim_output_dir, "antenna.cfg")
max_baseline = (
            ual.get_max_baseline_from_antenna_config(None, antennalist)
            * U.km)
beam_size = ual.estimate_alma_beam_size(
            central_freq, max_baseline, return_value=False
        )
snr = 1
beam_solid_angle = np.pi * (beam_size / 2) ** 2
cont_sens = cont_sens * U.mJy / (U.arcsec**2)
cont_sens_jy = (cont_sens * beam_solid_angle).to(U.Jy)
cont_sens = cont_sens_jy * snr
n_channels = t_channels
redshift = 0. 
type_ = 'point'
db_line = uas.read_line_emission_csv(
            os.path.join(main_path, "brightnes", "calibrated_lines.csv"),
            sep=",",
        )
central_freq = central_freq.value
band_range = band_range.value
source_freq = source_freq.value
cont_sent = cont_sens.value
freq_min = central_freq - band_range / 2
freq_max = central_freq + band_range / 2
print(freq_min, freq_max, band_range)
sed, flux_infrared, lum_infrared = sed_reading(
            type_,
            os.path.join(main_path, "brightnes"),
            cont_sens.value,
            freq_min,
            freq_max,
            False,
            None,
            None
        )

db_line["shifted_freq(GHz)"] = db_line["freq(GHz)"] / (1 + redshift)
n_avail = len(db_line[db_line["shifted_freq(GHz)"] >= freq_min])
print(f"{n_avail} out of {len(db_line)}")
  

268.76 270.63 1.8700000000000045
OK
268.76 270.63
102 out of 109


In [27]:
delta_v = 400 * U.km / U.s
c_km_s = c.to(U.km / U.s)
fwhms = (
            0.84
            * (db_line["shifted_freq(GHz)"].values * (delta_v / c_km_s) * 1e9)
            * U.Hz
        )
fwhms_GHz = fwhms.to(U.GHz).value
min_line, max_line = db_line["shifted_freq(GHz)"].values - fwhms_GHz, db_line["shifted_freq(GHz)"].values + fwhms_GHz



# 1. Apriamo il file di linee
# 2 Computa il redshift di tutte le linee rispetto alla source_freq
# 2 Selezioniamo o la prima linea utente (line mode yes) o la linea a piu basso redshfit (line mode no)
# 3. Controlliamo che la FWHM delle linee sia compatibile con la banda, in caso settiamo la fwhm alla banda
# 4. Ordiniamo le linee sulla base della distanza da questa linea
#  4.1 Controllo distanze centri  -> OK 
#   4.2 Controllo distanze estremi -> OK
# Se entrambi ok aggiungi riga e passa alla terza  
#   4.2 Invece Fallisce -> FAIL -> ridurre fwhm seconda linea 
#   4.2. Fallisce ancora FWHM = channel_size ----> Non posso aggiungere linee reali, invento lineaa. 
